<img src="resources/BerkeleySeal.png" width="150"/> <img src="resources/berkeleyischool-logo-blue.png" width="600"/>

# <center> Final project for course W261 - Machine Learning in Scale - Spring 2019</center>

### __Team 13:__ Clayton Leach, James Kajdasz,  Marcelo Queiroz, Peter Trenkwalder, Vishal Agarwal.

## Introduction:

## 1. Question Formulation

## 2. Algorithm Explanation

#### 2.1 Overview of Logistic Regression
Our group will apply a logistic regression model to the problem. We chose logistic regression because it is a very common and widely used algorithm, and the results of a logistic regression are relatively easy to communicate and interpret. Logistic regression is a classification algorithm that uses predictor variables (continuous or categorical) to classify an outcome variable in to one of two categories. The categories are arbitrarily given labels of '0' or '1', but can apply to any decision with two possible outcomes: cancerous/non-cancerous, spam/not-spam, fraudulent/legitimate... Logistic regression can be applied to a problem with more than two categories as well by creating a separate equation for each category: A/not A, B/not B, C/not C... The outcome variable is a probability (ranging 0 to 1) of group membership. The classification with the highest probability is the predicted category label.

#### 2.2 Logistic Regression Equation
Logistic regression aggregates the predictor variables similar to what is done in a standard linear regression. Each input $X_j$ is multiplied times a weight $\beta_j$ and each input/weight product $X_j \beta_j$ is added together. Or, in summarized form:  

$\displaystyle f(X)= \beta_0 + \Sigma X_j\beta_j$ 

In matrix algebra form, this can be written as $\displaystyle f(X)= \theta^TX$, where $\theta$ is a vector of weights (including $\beta_0$), and $X$ is a vector of inputs (with an input of 0 for $\beta_0$). The modification that logistic regression makes is to then embed the output of $\theta^TX$ in a new function $g(z)$ where $\displaystyle g(z)=\frac{1}{1+e^{-z}}$. To put all this together, $h_\theta (x) = g(\theta^Tx)$ where $g(z)=\frac{1}{1+e^{-z}}$. The function $g(z)$ is the sigmoid function, and it has the beneficial property of scaling all outputs between values of 0 and 1. We can write the equations above even more succinctly by substituting in $\theta^TX$ for $z$. Our final simplified equation is then: 

$\displaystyle h_\theta (x) = \frac{1}{1+e^{-\theta^TX}}$ 

We treat the value $h_\theta(x)$ as our estimate of probability that x is a member of category $y=1$. The probability that $y=0$ will then be $1 - h_\theta(x)$. Both probabilities will add to one. Recall that $h_\theta(x)$ ranges from 0 to 1 thanks to our application of the sigmoid function.       

#### 2.3 Cost Function
The weights of a logistic regression equation can vary, and so there must be a way to compare one hypothetical set of weights to another to judge if one model fits the data better than another. This is done by calculating the overall error of the model when attempting to predict $y$ to summarize model fit. The function that computes the error of the model is known as the cost or loss function. The goal of a model is to fit the data in such a way as to minimize the cost function. For a given set of weights $\theta$ attempting to predict a label $y_i$, the cost function of $h_\theta(X)$ can be quantified by simply squaring the errors where cost is $Cost \displaystyle (h_\theta (x_i), y_i) = \frac{1}{2} (h_\theta(X_i)-y_i)^2$. This is the standard cost function (known as squared loss) for linear regression. For logistic regression however, the squared loss function is not convex and has many local minima. Another alternative for the cost function must therefore be used. Alternatives include hinge loss, and logistic loss. The logistic loss function is used by Apache Spark, according to the [documentation](https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression). For logistic loss (log loss for short), we will take the negative log of the logistic regression output when the actual value of $y$ is 1. When the actual value of $y$ is 0, we will take the negative log of 1 minus the logistic regression output. Written mathematically: $\displaystyle Cost(h_\theta(x),y)= \begin{cases} -log(h_\theta(X)) & y=1\\-log(1-h_\theta(X)) & y=0\end{cases}$ The log loss function has some nice properties. When the logistic regression correctly predicts that $\hat{y}=1$ with a probability of 1, then $-log(1)=0$ and the loss function is zero, reflecting a perfect prediction. Similarly, if we (correctly) predict that $\hat{y}:0$ with a probability of 1, the cost function will be $-log(1-1)=0$. If however we make an incorrect prediction that $P(\hat{y}:0)=.999$, (and the corresponding probability $P(\hat{y}:1)=.001$) when in actuality $y=1$, then the log loss function will be $-log(.001)\approx3$, reflecting a higher amount of error. Note that we can't take the log of 0, so instead we use values of .999 and .001. As the correct prediction approaches a probability of 0, the log loss function will approach infinity. the prediction is $y=0$     

#### 2.4 Finding the Correct Model to Minimize the Cost Function
One could select logistic regression weights at random, and see if the new model is an improvement over the last model by evaluating the cost function, and continue iterating. This is inefficient however and it's unlikely we'd ever stumble across the best model by chance. It's better to have an algorithm that systematically moves us to better and better models. The cost function will, at it's minimum, have a slope of zero. If we take the derivative of the cost function to evaluate the slope, and then move our next iteration closer toward zero, we can eventually find a minimum of the cost function. The maximum of the cost function will also have a slope of zero, so we have to ensure we are moving in the correct direction to find a minimum. There are several different algorithms available that involve some variation of this approach, including gradient descent, Newton methods, and quasi-Newton methods. 

In gradient descent, we evaluate the first-order derivative of the cost function, which gives us the slope or gradient. We take our next step according to which direction results in the greatest negative change in gradient. The step-size (called the learning rate) is constant for each step and is predetermined by the user. After multiple iterations, we will eventually end up at some minimum. 

Newton's method uses additional information: the second-order derivative. The second-order derivative provides the function for a line that is tangent to the slope. This line can be traced back to the x-intercept to provide a new estimate for the next iteration of where the slope is zero. Newton's method has the advantage that it tends to converge in less iterations to the minimum value of the cost function than gradient descent. Still, Second-order methods like Newton's method tend to not be used for machine learning algorithms (see section 8.6 of this [link](http://www.deeplearningbook.org/contents/optimization.html)). The main reason they are not used is that they are computationally expensive. With $k$ parameters, the computation of Newton's method requires the inversion of a $k x k$ matrix, which has a run time of $O(k^3)$. The resultant computed matrix (known as the Hessian matrix) must be recalculated for every iteration. In addition, for a function that is not convex, Newton's method may converge on a local minima, not the global minima, although this can be alleviated somewhat with regularization strategies. 

To avoid the computationally expensive requirements of second-order methods like Newton's method, another class of algorithms exist called quasi-Newton methods. Quasi-Newton methods don't actually compute the Hessian matrix outright, but instead estimate the matrix in a way that is less computationally expensive but still returns reasonable approximations. A variety of techniques have been developed to estimate the Hessian matrix. One example is the Broyden-Fletcher-Goldfarb-Shanno (BFGS) algorithm, a variant of which we will use in our own logistic regression implementation.    

The BFGS algorithm, not to be confused with the [Canadian punk rock band](https://en.wikipedia.org/wiki/Bunchofuckingoofs), is an iterative method which is used to optimize the parameters of a function. It is part of a group of methods known as quasi-Newton methods that aim to approximate the second-order derivative instead of calculating it outright. To make its approximation of the Hessian matrix that is so time-consuming in the Newton method, the BFGS algorithm uses a hill-climbing technique that (similar to gradient descent) iterates through successive approximations of the matrix that are progressively more accurate. 

When working with a model that has a very large number of variables (greater than 1000), a modification of the BFGS algorithm called the Limited-memory BFGS (L-BFGS) algorithm is used. In the BFGS algorithm, the Hessian matrix that is being estimated is a $n x n$ matrix where $n$ is the number of parameters in the model. The L-BFGS algorithm condenses this dense representation to a smaller vector that approximates the full representation of variables. When working with data at scale, the Apache Spark [documentation](https://spark.apache.org/docs/latest/mllib-optimization.html#choosing-an-optimization-method) recommends the L-BFGS algorithm because it takes less iterations and is therefore quicker to converge than stochastic gradient descent. For this reason, our implementation of logistic regression will use the L-BFGS algorithm for model optimization. 

## 3. EDA and Discussion of Challenges

## 4. Algorithm Implementation

## 5. Application of Course Concepts